In [1]:
import pandas as pd

## 데이터 호출

In [14]:
import pyarrow as pa

file_path_train = '/data/ephemeral/level2-nlp-mrc-nlp-01/data/train_dataset/train/dataset.arrow'

In [15]:
with open(file_path_train, 'rb') as f:
    reader = pa.ipc.open_stream(f)
    table = reader.read_all()

df_train = table.to_pandas()

In [16]:
df_train.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [11]:
import json

file_path_wiki = '/data/ephemeral/level2-nlp-mrc-nlp-01/data/wikipedia_documents.json'

with open(file_path_wiki, 'r') as file:
    data = json.load(file)

In [ ]:
from datasets import load_dataset

corpus = list(set([example['context'] for example in df_train]))

In [31]:
from retrieval import SparseRetrieval

In [33]:
args={'dataset_name':"../data/train_dataset", 
      "model_name_or_path":"bert-base-multilingual-cased",
      "data_path":"../data", "context_path":"wikipedia_documents.json", "use_faiss":False}

In [27]:
from datasets import Dataset, concatenate_datasets, load_from_disk


org_dataset = load_from_disk(args['dataset_name'])
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
)  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

Flattening the indices:   0%|          | 0/3952 [00:00<?, ? examples/s]

Flattening the indices: 100%|██████████| 240/240 [00:00<00:00, 31060.29 examples/s]

**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})


In [34]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(args['model_name_or_path'], use_fast=False,)

retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path=args['data_path'],
    context_path=args['context_path'],
)


Lengths of unique contexts : 56737


In [35]:
import time
from contextlib import contextmanager

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")


In [36]:
def query_search_bulk(full_ds):
    with timer("bulk query by exhaustive search"):
        df = retriever.retrieve(full_ds)
        df["correct"] = df["original_context"] == df["context"]
        print(
            "correct retrieval result by exhaustive search",
            df["correct"].sum() / len(df),
        )
    return df

In [ ]:
def query_search_single(query):
    with timer("single query by exhaustive search"):
        scores, indices = retriever.retrieve(query)
    return scores, indices

In [38]:
retriever.get_sparse_embedding()
df=query_search_bulk(full_ds)

Build passage embedding


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(56737, 50000)
Embedding pickle saved.
[query exhaustive search] done in 34.072 s


Sparse retrieval: 100%|██████████| 4192/4192 [00:00<00:00, 9108.15it/s]

correct retrieval result by exhaustive search 0.2366412213740458
[bulk query by exhaustive search] done in 34.544 s


In [ ]:
retriever.get_relevant_doc_bulk(full_ds)

In [39]:
df.head()

,question,id,context,original_context,answers,correct
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 ...,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [235], 'text': ['하원']}",False
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}",True
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,현재는 대만 문제와 맞물려서 ‘하나의 중국’ 정책을 내세우고 있는 중화인민공화국에서...,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [510], 'text': ['백성']}",False
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,삼존불비상(三尊佛碑像)은 현재 동국대학교에 있는 것으로 충청남도 공주시 정안면에서 ...,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [625], 'text': ['중국']}",False
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,옹기장은 독과 항아리 등을 만드는 장인이다. 옹기장이 만든 옹기는 한민족이 다함께 ...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}",False
